<a href="https://colab.research.google.com/github/lorenzo-crippa/3M_NLP_ESS_2022/blob/main/Tutorial_Eleven_(R)_Causal_Inference_with_Latent_Treatments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal Inference with Latent Treatments

## Douglas Rice


In this notebook, we'll work with code and data from Fong and Grimmer's (2021) *AJPS* on "Causal Inference with Latent Treatments." Their study laid out an apporach that ``uses machine learning to discover the measured latent treatments.''  The goal is to understand how features of Donald Trump's tweets influenced citizen evaluations of the tweets. 


## Setup

In [1]:
install.packages("tidytext")
install.packages("texteffect")
install.packages("textdata")
install.packages("car")
install.packages("xtable")

library(tidytext)
library(texteffect)
library(textdata)
library(car)
library(xtable)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘SnowballC’, ‘hunspell’, ‘janeaustenr’, ‘tokenizers’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘sp’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘maptools’, ‘lme4’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: MASS

Loading required package: boot

Loading required package: ggplot2

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:boot’:

    logit




# Data

For this, we'll use a corpus of Donald Trump's tweets and associated metadata from Fong & Grimmer. The data comes from YouGov's TweetIndex data from February 4, 2017 to October 31, 2017. YouGov presents citizens with tweets from the then-president and asked them to rate the tweet on a five point scale from from ``Great'' to ``Terrible''. I've posted the dataset on my Google Drive and you access it as follows. 

In [2]:
system("gdown --id 1EgTgqn3o6edyCH9TUxzErw5iFdTPGHdb")
dat <- read.csv("trumpdt.csv")

In [3]:
head(dat)

,score,Gind,Gdem,Grep,saved,votes,companies,heading,enjoy,leaders,⋯,statement,washington,biggest,north,signed,record,stories,problem,chance,repeal
,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0.04735535,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,-77.82785352,0,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,85.10157541,0,0,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,-31.27180013,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,-129.17740290,0,1,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,78.85985452,0,0,1,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [6]:
dim(dat) # 4509 tweets, 300 words (4 variables refer to something else)

[1] 4509  304

## Set the Data Up

We begin by dividing the data into the rating (Y), a field that features the party ID indicator for the survey respondents evaluation (G), and a data frame of word counts (X). 

In [4]:
Y <- dat[,1]
G <- dat[,2:4]
X <- dat[,5:ncol(dat)]

# take a peak at the set up of the tweets
head(X)

,saved,votes,companies,heading,enjoy,leaders,governor,men,china,luther,⋯,statement,washington,biggest,north,signed,record,stories,problem,chance,repeal
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [8]:
# how many mentions of companies?
sum(X$companies) # 51 mentions of companies

[1] 51

In [9]:
# how many tweets mention companies?
sum(X$companies > 0) # 51 tweets

[1] 51

In [10]:
max(X$companies) # 1, indeed

[1] 1

Next, we create training and test sets. The reason to do so here follows from earlier work by Fong & Grimmer (2016); the underlying idea is that by creating a ``training'' and ``test'' set we can separate the process of estimating treatment from the process of estimating effects, and thereby avoid cross-contamination in our causal framework.

One challenge here is that our analysis is at the tweet-respondent level; the same tweets are often presented to  Republican, Democrat, and Independent survey respondents. We'd like to ensure that we maintain the tweet groupings across the training and test sets. Therefore, random assignment to the training and test set happens at the tweet level, rather than at the individual level.


In [12]:
nrow(X)/3 # 1503 tweets

[1] 1503

In [13]:
nrow(X)/3*.5 # 751.5 sampled

[1] 751.5

In [15]:
training.tweets <- sample(1:(nrow(X)/3), nrow(X)/3*.5)
head(training.tweets)

[1]  474  650  836  397 1169  633

In [16]:
train.ind <- c()
for (i in 1:length(training.tweets)){
  train.ind <- c(train.ind, 3*(training.tweets[i]-1)+(1:3))
}

# Parameter Search

Fong and Grimmer develop the supervised Indian Buffet Process for *discovering* latent treatments from text. As background, the Indian Buffet Process is one of many different unsupervised approaches for recovering the latent structure responsible for generating observed data (think LDA and other topic model type processes). The primary advantage of the Indian Buffet Process here is that it produces a binary topic vector, which makes inferring treatment more straightforward than continuous measures. The **supervised** Indian Buffet Process developed by Fong and Grimmer goes a step further by incorporating in the estimation of the latent features both the text of the tweets (X) *and* the response (Y).  

As noted above, we separate out the data into training and test sets. We begin then by training the sIBP on the training data. We run the the sIBP across different parameters, allowing us to search for a specification that best reflects the data. 

This takes about 8 minutes to run. 

In [17]:
sibp.search <- sibp_param_search(X, Y, K = 5, alphas = c(3,4), sigmasq.ns = c(0.5, 1), 
                                 iters = 5, train.ind = train.ind, G = G, seed = 12082017)



[1] 3
[1] 0.5
[1] 1
[1] 4
[1] 0.5
[1] 1


## Select a Set

Next, we need to select one of these as the set to analyze. We should select the set *before* doing any of the subsequent analysis. We'll follow Fong & Grimmer, and work with the following set. 

In [18]:
sibp.fit <- sibp.search[["3"]][["0.5"]][[1]]


We can see some more details about the set using the top words functionality as below. This is also helpful if you were interested in poking around with different models. **The most important note here is that you should not proceed to inference until you are comfortable with the model you have selected.**

In [19]:
xtable(sibp_top_words(sibp.fit, colnames(X), verbose = TRUE))


[1] "Frequency of treatments: "
[1] 354.1390 200.1528 266.0172 173.9976 113.9997
[1] "Relation between top words and treatments"
          [,1]     [,2]     [,3]     [,4]     [,5]
[1,] 0.9547454 1.228397 1.301900 2.019712 2.029291
[2,] 0.9547454 1.170223 1.301900 1.791365 1.972772
[3,] 0.7801361 1.165447 1.110547 1.723626 1.903796
[4,] 0.7108009 1.120314 1.064215 1.620610 1.858840
[5,] 0.6632524 1.105227 1.025672 1.132865 1.701299
[6,] 0.6632524 1.103123 0.988239 1.072602 1.671516


1,2,3,4,5
<chr>,<chr>,<chr>,<chr>,<chr>
prime,high,rico,senators,flotus
minister,nytimes,puerto,repeal,cuts
melania,cnn,stock,replace,luther
flotus,nfl,market,republican,strange
rico,stock,players,healthcare,alabama
puerto,luther,nfl,obamacare,honored
nytimes,premiums,anthem,crooked,melania
united,strange,tax,dead,reform
conference,market,flag,dems,tax


# Inference

The above creates a mapping between texts and latent treatments. Therefore, we turn next to leveraging that mapping to infer values of our latent treatments within the test set. 

In [20]:
X.test <- t(apply(X[sibp.fit$test.ind,], 1, function(x) (x - sibp.fit$meanX)/sibp.fit$sdX))
nu.test <- infer_Z(sibp.fit, X)
Z.train <- matrix(as.numeric(sibp.fit$nu >= 0.5), ncol = 5)
Z.test <- matrix(as.numeric(nu.test >= 0.5), ncol = 5)



Now that we have our mapping, we need to turn back towards the party subsets that we are primarily interested in. Therefore, we create a data frame with respondent party indicators.  

In [21]:
dat2 <- data.frame(Y[sibp.fit$test.ind], G[sibp.fit$test.ind,])
colnames(dat2) <- c("Y", "ind", "dem", "rep")

In [22]:
head(dat2)

,Y,ind,dem,rep
,<dbl>,<int>,<int>,<int>
7,-13.02126,1,0,0
8,-109.62065,0,1,0
9,107.56427,0,0,1
16,10.10317,1,0,0
17,-17.31307,0,1,0
18,106.48820,0,0,1


Next, we add in our estimates for the latent treatments inferred above. 

In [23]:
dat2$Z1 <- Z.test[,1]
dat2$Z2 <- Z.test[,2]
dat2$Z3 <- Z.test[,3]
dat2$Z4 <- Z.test[,4]
dat2$Z5 <- Z.test[,5]

In [24]:
head(dat2)

,Y,ind,dem,rep,Z1,Z2,Z3,Z4,Z5
,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7,-13.02126,1,0,0,0,0,0,0,0
8,-109.62065,0,1,0,0,0,0,0,0
9,107.56427,0,0,1,0,0,0,0,0
16,10.10317,1,0,0,0,0,0,0,0
17,-17.31307,0,1,0,0,0,0,0,0
18,106.48820,0,0,1,0,0,0,0,0


### Sentiment 

Fong & Grimmer include in their model  sentiment scores estimated via a dictionary approach (AFINN), with tweets dichotomized into positive or negative.  


In [ ]:
system("gdown --id 1EkBHsr1aRltyA0KveZFgrLhiTo7G4OvQ")
afinn <- read.csv("afinn.csv")

start<- as.matrix(afinn)
use0 <- match(colnames(X), start[,2])
use <- use0[which(!is.na(use0))]
use_col<- which(is.na(match(colnames(X), start[,2]))==F)
sents<- as.matrix(X[sibp.fit$test.ind,use_col])%*%as.numeric(start[use,3])
dat2$sents <- I(sents > 0)




## Results

Notice what we now have. The `dat2` dataframe features the rating, a set of latent of treatment indicators, the party of the respondent, and the sentiment (positive or negative) of the tweet. Recall that the goal of the exercise was to understand the effect of different features on evaluations of Donald Trump's tweets by party. Therefore, Fong & Grimmer estimate three models (one for each of Democratic, Republican, and Independent respondents). 

In [ ]:
# Democrats
summary(lm(Y ~ Z1 + Z2 + Z3 + Z4 + Z5, data = dat2, subset = which(dat2$dem == 1)))




Call:
lm(formula = Y ~ Z1 + Z2 + Z3 + Z4 + Z5, data = dat2, subset = which(dat2$dem == 
    1))

Residuals:
   Min     1Q Median     3Q    Max 
-71.58 -35.72  -5.23  30.32 123.94 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -81.987      1.704 -48.110  < 2e-16 ***
Z1           -27.757      5.593  -4.963 8.59e-07 ***
Z2           -12.843      7.470  -1.719    0.086 .  
Z3            46.701      6.921   6.748 3.02e-11 ***
Z4            -3.102     10.510  -0.295    0.768    
Z5           -15.820      9.699  -1.631    0.103    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 43.03 on 746 degrees of freedom
Multiple R-squared:  0.08381,	Adjusted R-squared:  0.07767 
F-statistic: 13.65 on 5 and 746 DF,  p-value: 9.125e-13


In [ ]:
## Republicans
summary(lm(Y ~ Z1 + Z2 + Z3 + Z4 + Z5, data = dat2, subset = which(dat2$rep == 1)))



Call:
lm(formula = Y ~ Z1 + Z2 + Z3 + Z4 + Z5, data = dat2, subset = which(dat2$rep == 
    1))

Residuals:
    Min      1Q  Median      3Q     Max 
-91.376 -16.889  -1.232  17.066  65.116 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   97.769      1.012  96.642  < 2e-16 ***
Z1            -9.047      3.320  -2.725 0.006577 ** 
Z2            -3.971      4.435  -0.896 0.370802    
Z3            15.690      4.109   3.819 0.000145 ***
Z4            -3.533      6.239  -0.566 0.571354    
Z5            -8.560      5.757  -1.487 0.137514    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 25.55 on 746 degrees of freedom
Multiple R-squared:  0.03007,	Adjusted R-squared:  0.02357 
F-statistic: 4.625 on 5 and 746 DF,  p-value: 0.0003651


In [ ]:
## Independents
summary(lm(Y ~ Z1 + Z2 + Z3 + Z4 + Z5, data = dat2, subset = which(dat2$ind == 1)))



Call:
lm(formula = Y ~ Z1 + Z2 + Z3 + Z4 + Z5, data = dat2, subset = which(dat2$ind == 
    1))

Residuals:
    Min      1Q  Median      3Q     Max 
-76.807 -26.367  -3.346  22.566  98.127 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.05033    1.32820  -0.038   0.9698    
Z1          -18.43701    4.35876  -4.230 2.63e-05 ***
Z2           -5.82438    5.82207  -1.000   0.3174    
Z3           30.07931    5.39429   5.576 3.44e-08 ***
Z4           -6.88933    8.19177  -0.841   0.4006    
Z5          -15.15926    7.55895  -2.005   0.0453 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 33.54 on 746 degrees of freedom
Multiple R-squared:  0.0636,	Adjusted R-squared:  0.05732 
F-statistic: 10.13 on 5 and 746 DF,  p-value: 2.1e-09


We can see from the above a few interesting dynamics. First, Z5 (or the fifth latent treatment) seems to be associated with a significant shift in evaluations only among independents. Second, Z1 and Z3 are consistently across party influencing evaluations, and in directions (though not magnitude) that are consistent across party. 

Fong & Grimmer go a step further and estimate the models with a sentiment control. Interestingly, with the sentiment control included, thereby addressing another potentially unobserved confounder. The differences in treatment effects across party narrow.

In [ ]:
summary(lm(Y ~ Z1 + Z2 + Z3 + Z4 + Z5 + sents, data = dat2, subset = which(dat2$dem == 1)))
summary(lm(Y ~ Z1 + Z2 + Z3 + Z4 + Z5 + sents, data = dat2, subset = which(dat2$ind == 1)))
summary(lm(Y ~ Z1 + Z2 + Z3 + Z4 + Z5 + sents, data = dat2, subset = which(dat2$rep == 1)))



Call:
lm(formula = Y ~ Z1 + Z2 + Z3 + Z4 + Z5 + sents, data = dat2, 
    subset = which(dat2$dem == 1))

Residuals:
   Min     1Q Median     3Q    Max 
-80.85 -33.05  -7.21  29.19 125.54 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -92.783      2.122 -43.728  < 2e-16 ***
Z1           -26.826      5.372  -4.994 7.38e-07 ***
Z2           -11.148      7.177  -1.553   0.1208    
Z3            36.813      6.761   5.445 7.04e-08 ***
Z4            -1.274     10.096  -0.126   0.8996    
Z5           -18.569      9.320  -1.992   0.0467 *  
sentsTRUE     24.651      3.084   7.994 4.99e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 41.33 on 745 degrees of freedom
Multiple R-squared:  0.1562,	Adjusted R-squared:  0.1494 
F-statistic: 22.98 on 6 and 745 DF,  p-value: < 2.2e-16



Call:
lm(formula = Y ~ Z1 + Z2 + Z3 + Z4 + Z5 + sents, data = dat2, 
    subset = which(dat2$ind == 1))

Residuals:
    Min      1Q  Median      3Q     Max 
-78.027 -24.147  -3.551  22.244  95.741 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   -8.576      1.652  -5.192 2.69e-07 ***
Z1           -17.701      4.182  -4.233 2.60e-05 ***
Z2            -4.485      5.587  -0.803   0.4224    
Z3            22.270      5.263   4.231 2.61e-05 ***
Z4            -5.446      7.860  -0.693   0.4886    
Z5           -17.330      7.256  -2.389   0.0172 *  
sentsTRUE     19.469      2.401   8.110 2.09e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 32.17 on 745 degrees of freedom
Multiple R-squared:  0.1396,	Adjusted R-squared:  0.1326 
F-statistic: 20.14 on 6 and 745 DF,  p-value: < 2.2e-16



Call:
lm(formula = Y ~ Z1 + Z2 + Z3 + Z4 + Z5 + sents, data = dat2, 
    subset = which(dat2$rep == 1))

Residuals:
    Min      1Q  Median      3Q     Max 
-88.691 -15.715  -0.689  15.632  62.896 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   92.507      1.277  72.437  < 2e-16 ***
Z1            -8.594      3.233  -2.658  0.00803 ** 
Z2            -3.145      4.320  -0.728  0.46683    
Z3            10.871      4.069   2.672  0.00771 ** 
Z4            -2.642      6.077  -0.435  0.66380    
Z5            -9.900      5.610  -1.765  0.07802 .  
sentsTRUE     12.015      1.856   6.473 1.74e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 24.87 on 745 degrees of freedom
Multiple R-squared:  0.08172,	Adjusted R-squared:  0.07432 
F-statistic: 11.05 on 6 and 745 DF,  p-value: 8.018e-12
